In [3]:
from soap import * 
import os

from model import MOFSetTransformerTrainer, MOFDataManager, ModifiedAggregateFunction

# training_pipeline.py

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch_geometric.nn.aggr import SetTransformerAggregation
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import os
from typing import Tuple, Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

from models import SetTransformer

import wandb

In [6]:
model  = SetTransformer(dim_input = 484, num_outputs = 1, dim_output = 1, num_inds=32, dim_hidden=256, num_heads=4)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [8]:
torch.load("optimized_models_fast60.pth")

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [1]:

model.load_state_dict(torch.load("optimized_models_fast60.pth"))

NameError: name 'model' is not defined

In [2]:
dir = "../CIF_files"
filenames = os.listdir(dir)

In [3]:
data_manager = MOFDataManager(
        folder_path='../CIF_files',  # Adjust path as needed
        target_csv="../id_labels.csv",  # Your target file
        val_size=0.0,   # 20% for validation
        test_size=0.2,  # 20% for test (60% remains for training)
        random_state=42
    )


Loading targets from: ../id_labels.csv
CSV columns: ['id', 'label']
CSV shape: (3089, 2)
First few rows:
                                               id      label
0  DB5-hypotheticalMOF_17652_0_0_1_21_9_7_repeat   13.791591
1             DB0-m2_o8_o23_f0_pcu.sym.80_repeat    3.786996
2         DB0-m29_o90_o1500_f0_pts.sym.31_repeat    9.382537
3             DB0-m3_o48_o25_f0_fsc.sym.3_repeat   11.650365
4             DB0-m2_o1_o9_f0_nbo.sym.104_repeat    1.412915
Using 'id' column for filenames and 'label' column for targets
Loaded 3089 target values
Target range: 0.006 to 37.706
Example filename mappings:
  'DB5-hypotheticalMOF_17652_0_0_1_21_9_7_repeat.cif' -> 13.79159108
  'DB0-m2_o8_o23_f0_pcu.sym.80_repeat.cif' -> 3.786995708
  'DB0-m29_o90_o1500_f0_pts.sym.31_repeat.cif' -> 9.382537337
Total CIF files found: 3089
CIF files with targets: 3089
Data split:
  Train files: 2471 (80.0%)
  Test files: 618 (20.0%)
✓ Verified no data leakage between splits


In [4]:
data_manager

In [5]:
trainer = MOFSetTransformerTrainer(
        data_manager=data_manager,
        aggregator_params={
            'num_seed_points': 1,        # Current: 3, try more seeds
            'num_encoder_blocks': 1,     # Current: 2, try more layers
            'num_decoder_blocks': 1,     # Current: 1, try more
            'heads': 4,                  # Current: 4, try more attention heads
            'dropout': 0.3,              # Current: 0.1, try higher dropout
        },
        loss_metric='mae' 
    )

In [6]:
trainer.determine_all_soap_columns()

Determining all unique SOAP columns across ENTIRE dataset...
Processing ALL 3089 files to find unique columns...
Processed 0/3089 files, found 0 unique columns so far...


Processed 100/3089 files, found 188 unique columns so far...
Processed 200/3089 files, found 236 unique columns so far...
Processed 300/3089 files, found 282 unique columns so far...
Processed 400/3089 files, found 298 unique columns so far...
Processed 500/3089 files, found 300 unique columns so far...
Processed 600/3089 files, found 312 unique columns so far...
Processed 700/3089 files, found 318 unique columns so far...
Processed 800/3089 files, found 332 unique columns so far...
Processed 900/3089 files, found 332 unique columns so far...
Processed 1000/3089 files, found 352 unique columns so far...
Processed 1100/3089 files, found 354 unique columns so far...
Processed 1200/3089 files, found 372 unique columns so far...
Processed 1300/3089 files, found 386 unique columns so far...
Processed 1400/3089 files, found 394 unique columns so far...
Processed 1500/3089 files, found 420 unique columns so far...
Processed 1600/3089 files, found 424 unique columns so far...
Processed 1700/30

['Ag-Ag_1',
 'Ag-Ag_2',
 'Ag-C_1',
 'Ag-C_2',
 'Ag-Cr_1',
 'Ag-Cr_2',
 'Ag-O_1',
 'Ag-O_2',
 'Al-Al_1',
 'Al-Al_2',
 'Al-C_1',
 'Al-C_2',
 'Al-F_1',
 'Al-F_2',
 'Al-H_1',
 'Al-H_2',
 'Al-N_1',
 'Al-N_2',
 'Al-O_1',
 'Al-O_2',
 'Al-P_1',
 'Al-P_2',
 'Al-S_1',
 'Al-S_2',
 'Br-Br_1',
 'Br-Br_2',
 'Br-C_1',
 'Br-C_2',
 'Br-Cd_1',
 'Br-Cd_2',
 'Br-Ce_1',
 'Br-Ce_2',
 'Br-Cl_1',
 'Br-Cl_2',
 'Br-Co_1',
 'Br-Co_2',
 'Br-Cu_1',
 'Br-Cu_2',
 'Br-F_1',
 'Br-F_2',
 'Br-Fe_1',
 'Br-Fe_2',
 'Br-H_1',
 'Br-H_2',
 'Br-La_1',
 'Br-La_2',
 'Br-Li_1',
 'Br-Li_2',
 'Br-N_1',
 'Br-N_2',
 'Br-O_1',
 'Br-O_2',
 'Br-Pd_1',
 'Br-Pd_2',
 'Br-S_1',
 'Br-S_2',
 'Br-Si_1',
 'Br-Si_2',
 'Br-Ti_1',
 'Br-Ti_2',
 'Br-V_1',
 'Br-V_2',
 'Br-Zn_1',
 'Br-Zn_2',
 'C-C_1',
 'C-C_2',
 'C-Ca_1',
 'C-Ca_2',
 'C-Cd_1',
 'C-Cd_2',
 'C-Ce_1',
 'C-Ce_2',
 'C-Cl_1',
 'C-Cl_2',
 'C-Co_1',
 'C-Co_2',
 'C-Cr_1',
 'C-Cr_2',
 'C-Cu_1',
 'C-Cu_2',
 'C-Dy_1',
 'C-Dy_2',
 'C-Er_1',
 'C-Er_2',
 'C-Eu_1',
 'C-Eu_2',
 'C-F_1',
 'C-F_2',
 'C-

In [7]:
 results = trainer.train(
        epochs=5,
        learning_rate=0.01,  # Lower LR for end-to-end training
        patience=100,
        batch_size=100,      # Process 100 files per epoch
        verbose=True
    )

Initialized SetTransformerAggregation with:
  SOAP dimension: 484
  Unique columns: 484
  channels: 484
  num_seed_points: 1
  num_encoder_blocks: 1
  num_decoder_blocks: 1
  heads: 4
  concat: True
  layer_norm: True
  dropout: 0.3
Initialized prediction head with input dim: 484
Starting end-to-end training for 5 epochs...
Train files: 2471, Test files: 618
Using batch size: 100
